# Simple Bitcoin Sentements Measurement 

## Import Libraries 

In [79]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import re
import string
from nltk.stem import PorterStemmer
import nltk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import twint
import pandas as pd
import sys
sys.path.append("twint/")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [80]:
import nest_asyncio
nest_asyncio.apply()

## Using twint Library, we will scrap twitter searching for Tweets related to Bitcoin 

In [81]:
c = twint.Config()
c.Search = ['Bitcoin']# topic
c.Limit = 5000   # number of Tweets to scrape
# c.Pandas_clean = 'pandas-clean' /  optional 
c.Pandas = True
 # if you like to scrap from specific time uncoment and change these 
# c.Since = '2021-10-15' 
# c.Until = '2021-10-20'

c.Store_csv = True       # store tweets in a csv file
c.Output = "Bitcoin2.csv"     # path to csv file


## Begin the search 

In [ ]:
twint.run.Search(c)

## Organize the dataframe columns`

In [7]:
def available_columns():
    return twint.output.panda.Tweets_df.columns

In [8]:
def twint_to_pandas(columns):
    return twint.output.panda.Tweets_df[columns]

In [9]:
available_columns()

Index(['id', 'conversation_id', 'created_at', 'date', 'timezone', 'place',
       'tweet', 'language', 'hashtags', 'cashtags', 'user_id', 'user_id_str',
       'username', 'name', 'day', 'hour', 'link', 'urls', 'photos', 'video',
       'thumbnail', 'retweet', 'nlikes', 'nreplies', 'nretweets', 'quote_url',
       'search', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest'],
      dtype='object')

## Get the columns of interest 

In [10]:
df_pd = twint_to_pandas(["date", "username", "tweet", "hashtags", "nlikes"])
df_pd

,date,username,tweet,hashtags,nlikes
0,2021-10-22 12:52:44,sbbot123,Crypto Prices (USD/₿) Bitcoin ➦ $63227.6859➶ ...,"[crypto, bitcoin, ethereum, binance, cardano, ...",0
1,2021-10-22 12:52:42,BlogiBossaPL,@PaweIZawadzki @burns66675164 Powód zasadniczy...,[],0
2,2021-10-22 12:52:42,Bokiko_io,Weekly Fees: Sum of total fees spent to use a ...,"[btc, eth]",0
3,2021-10-22 12:52:40,DerekTMcKinney,🤓🤔👍 #knownews #AMBCrypto #bitcoin #bnb #Crypto...,"[knownews, ambcrypto, bitcoin, bnb, cryptocurr...",0
4,2021-10-22 12:52:36,Bitcoinesque1,@firma_firm @A4lfr32 @SoulsOfEarth @newjerseyw...,[],0
...,...,...,...,...,...
1006,2021-10-22 12:36:22,TOVANLEN2,@FollowTop_nft Thank you for the opportunity. ...,"[airdrop, airdrops, airdropinspector, followpr...",0
1007,2021-10-22 12:36:21,bmurphypointman,#marketing #bitcoin #tumblr #twitter #facebook...,"[marketing, bitcoin, tumblr, twitter, facebook...",0
1008,2021-10-22 12:36:21,bmurphypointman,#affilate #earnmoney #affiliateprograms #affil...,"[affilate, earnmoney, affiliateprograms, affil...",0
1009,2021-10-22 12:36:20,bmurphypointman,#linkedin #shoppingtime #twitter #facebook #in...,"[linkedin, shoppingtime, twitter, facebook, in...",0


## We will use only tweet column here 

In [35]:
tweet= df_pd['tweet']
tweet

0       Crypto Prices (USD/₿)  Bitcoin ➦ $63227.6859➶ ...
1       @PaweIZawadzki @burns66675164 Powód zasadniczy...
2       Weekly Fees: Sum of total fees spent to use a ...
3       🤓🤔👍 #knownews #AMBCrypto #bitcoin #bnb #Crypto...
4       @firma_firm @A4lfr32 @SoulsOfEarth @newjerseyw...
                              ...                        
1006    @FollowTop_nft Thank you for the opportunity. ...
1007    #marketing #bitcoin #tumblr #twitter #facebook...
1008    #affilate #earnmoney #affiliateprograms #affil...
1009    #linkedin #shoppingtime #twitter #facebook #in...
1010    #tumblr #shoppingtime #twitter #facebook #inst...
Name: tweet, Length: 1011, dtype: object

## Show some results for tweet 

In [56]:
n_tweets = 0
Positive = 0
Negative = 0
Neutral = 0

sid_obj = SentimentIntensityAnalyzer()

for col in tweet.iteritems():
    n_tweets+=1


    sentiment_dict = sid_obj.polarity_scores(col[1])
     
    # decide sentiment as positive, negative and neutral
    if sentiment_dict['compound'] >= 0.05 :
        #print("Positive")
        Positive+=1
 
    elif sentiment_dict['compound'] <= - 0.05 :
        #print("Negative")
        Negative+=1
 
    else :
        #print("Neutral")
        Neutral+=1

print('Totale Number of tweets = ',n_tweets)  
print('Postive tweets = ',Positive, ' ||| Per',int(Positive*100.0 / n_tweets),'%' )
print('Negative Tweets = ',Negative, ' ||| Per',int(Negative*100.0 / n_tweets),'%')
print('Neutral tweets = ',Neutral, ' ||| Per',int(Neutral*100.0 / n_tweets),'%')
print()

Totale Number of tweets =  1011
Postive tweets =  435  ||| Per 43 %
Negative Tweets =  157  ||| Per 15 %
Neutral tweets =  419  ||| Per 41 %



In [59]:
import re,string

def strip_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text

def strip_all_entities(text):
    entity_prefixes = ['@','#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)


tests = [
    "@peter I really love that shirt at #Macy. http://bet.ly//WjdiW4",
    "@shawn Titanic tragedy could have been prevented Economic Times: Telegraph.co.ukTitanic tragedy could have been preve... http://bet.ly/tuN2wx",
    "I am at Starbucks http://4sh.com/samqUI (7419 3rd ave, at 75th, Brooklyn)"]

for t in tests:
    a = strip_all_entities(strip_links(t))
    print(a)

I really love that shirt at
Titanic tragedy could have been prevented Economic Times Telegraph co ukTitanic tragedy could have been preve
I am at Starbucks 7419 3rd ave at 75th Brooklyn


In [65]:
clean_tweets = []
for col in tweet.iteritems():
    clean = strip_all_entities(col[1])
    clean_tweets.append(clean)
    
df_tweet = pd.DataFrame (clean_tweets, columns = ['tweets'])    

In [74]:
df_tweet

,tweets
0,Crypto Prices USD ₿ Bitcoin ➦ 63227 6859➶ Ethe...
1,Powód zasadniczy zmieniły się zasady gry bo bi...
2,Weekly Fees Sum of total fees spent to use a p...
3,🤓🤔👍 ArbiSmart’s token provides an investment o...
4,firm You see the purpose of bitcoin is to make...
...,...
1006,nft Thank you for the opportunity I hope this ...
1007,on orders over 99 at https t co FsGWUaZyeH
1008,Check my website for https t co vBHLD11zwQ
1009,Enjoy 10 off Orders of 50 Use code IR10 https ...


In [78]:
n_tweets = 0
Positive = 0
Negative = 0
Neutral = 0

sid_obj = SentimentIntensityAnalyzer()

for col in clean_tweets:
    n_tweets+=1


    sentiment_dict = sid_obj.polarity_scores(col)
     
    # decide sentiment as positive, negative and neutral
    if sentiment_dict['compound'] >= 0.05 :
        #print("Positive")
        Positive+=1
 
    elif sentiment_dict['compound'] <= - 0.05 :
        #print("Negative")
        Negative+=1
 
    else :
        #print("Neutral")
        Neutral+=1

print('Totale Number of tweets = ',n_tweets)  
print('Postive tweets = ',Positive, ' ||| Per',int(Positive*100.0 / n_tweets),'%' )
print('Negative Tweets = ',Negative, ' ||| Per',int(Negative*100.0 / n_tweets),'%')
print('Neutral tweets = ',Neutral, ' ||| Per',int(Neutral*100.0 / n_tweets),'%')
print()

Totale Number of tweets =  1011
Postive tweets =  419  ||| Per 41 %
Negative Tweets =  165  ||| Per 16 %
Neutral tweets =  427  ||| Per 42 %



In [86]:
from textblob import TextBlob
obj = TextBlob( "Weekly Fees Sum of total fees spent to use a p..." )

print(obj.sentiment)

Sentiment(polarity=-0.05, subjectivity=0.425)


In [95]:
n_tweets = 0
Positive = 0
Negative = 0
Neutral = 0


for col in clean_tweets:
    n_tweets+=1


    sentiment_dict = TextBlob(col)
    #print(sentiment_dict.sentiment[1])
     
    # decide sentiment as positive, negative and neutral
    if sentiment_dict.sentiment[0] >= 0.05 :
        #print("Positive")
        Positive+=1
 
    elif sentiment_dict.sentiment[0] <= - 0.05 :
        #print("Negative")
        Negative+=1
 
    else :
        #print("Neutral")
        Neutral+=1

print('Totale Number of tweets = ',n_tweets)  
print('Postive tweets = ',Positive, ' ||| Per',int(Positive*100.0 / n_tweets),'%' )
print('Negative Tweets = ',Negative, ' ||| Per',int(Negative*100.0 / n_tweets),'%')
print('Neutral tweets = ',Neutral, ' ||| Per',int(Neutral*100.0 / n_tweets),'%')
print()

Totale Number of tweets =  1011
Postive tweets =  383  ||| Per 37 %
Negative Tweets =  67  ||| Per 6 %
Neutral tweets =  561  ||| Per 55 %

